In [2]:
# visualization
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
fe = fm.FontEntry(
    fname=r'/usr/share/fonts/truetype/nanum/NanumGothic.ttf', # ttf 파일이 저장되어 있는 경로
    name='NanumBarunGothic')                        # 이 폰트의 원하는 이름 설정
fm.fontManager.ttflist.insert(0, fe)              # Matplotlib에 폰트 추가
plt.rcParams.update({'font.size': 10, 'font.family': 'NanumBarunGothic'}) # 폰트 설정
plt.rc('font', family='NanumBarunGothic')
import seaborn as sns

# utils
import pandas as pd
import numpy as np
from tqdm import tqdm
import pickle
import warnings;warnings.filterwarnings('ignore')

# Model
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics

import eli5
from eli5.sklearn import PermutationImportance

In [3]:
# 필요한 데이터를 load 하겠습니다. 경로는 환경에 맞게 지정해주면 됩니다.
train_path = '/data/ephemeral/home/Upstage_ML_Competition01/data/train_latlon_complete.csv'
test_path  = '/data/ephemeral/home/Upstage_ML_Competition01/data/test_latlon_complete.csv'
codebook_path = '/data/ephemeral/home/Upstage_ML_Competition01/data/codebook.csv'
train = pd.read_csv(train_path)
test = pd.read_csv(test_path)
codebook = pd.read_csv(codebook_path)

In [4]:
# train/test 구분을 위한 칼럼을 하나 만들어 줍니다.
train['is_test'] = 0
test['is_test'] = 1
concat = pd.concat([train, test])     # 하나의 데이터로 만들어줍니다.

In [5]:
# codebook에 있는 변수명들로 변경해줍시다.
variables = codebook.set_index('한글 변수명').to_dict()['영문 변수명'] # 코드북에 있는 한글 변수명 column을 키로 영문 변수명 column을 value로 한 딕셔너리타입 변수 생성
concat.rename(columns=variables, inplace=True) # column의 이름 변경

In [6]:
concat.columns

Index(['ADRES', 'LOT_NO', 'MAIN_NO', 'SUB_NO', 'APT_NM', 'EXCL_AREA_SQM',
       'CONTR_YEAR_MONTH', 'CONTR_DAY', 'FLOOR', 'BUILD_YEAR', 'ADRES_DORO',
       'CANCEL_REASON_DATE', 'REGIST_APPL_DATE', 'TRADE_TYPE',
       'AGNCY_LOCATION', 'CODEAPTNM', 'TELNO', 'FXNUM', 'CN_APT', 'HSHLDR_TY',
       'GNRL_MANAGECT_MANAGE_STLE', 'CRRDPR_TY', 'HEAT_MTHD', 'ALL_DONG_CO',
       'ALL_HSHLD_CO', 'CO_WO', 'CO_EX', 'USE_INSPCT_DE', 'TOTAR', 'PRIVAREA',
       'MANAGECT_LEVY_AR', 'KAPTMPAREA60', 'KAPTMPAREA85', 'KAPTMPAREA135',
       'KAPTMPAREA136', 'HMPG', 'API_INSERT_DATE', 'API_UPDATE_DATE',
       'EMPLYMINSRNC_MANAGE_NO', 'EXPENSCTMANAGESTLE',
       'HSHLD_ELCTY_CNTRCT_MTH', 'CLN_CTMANAGESTLE', 'BU_AR', 'CNT_PA',
       'GUBUN', 'USE_CONFM_DE', 'USE_TY', 'MANUAL_INPUT', 'X_CODE', 'Y_CODE',
       'USE_RQSTDT', 'target', 'is_test'],
      dtype='object')

In [7]:
coordinate = concat[['ADRES', 'APT_NM','X_CODE', 'Y_CODE', 'USE_RQSTDT', 'target', 'is_test']]

In [8]:
coordinate['GU'] = coordinate['ADRES'].map(lambda x : x.split()[1])
coordinate['DONG'] = coordinate['ADRES'].map(lambda x : x.split()[2])
del coordinate['ADRES']

In [9]:
concat["HAN_VIEW"] = coordinate[(coordinate["APT_NM"]=="서울숲리버뷰자이") | (coordinate["APT_NM"]=="힐스테이트서울숲리버") | (coordinate["APT_NM"]=="아크로리버뷰신반포") | 
                    (coordinate["APT_NM"]=="경동윈츠리버") | (coordinate["APT_NM"]=="리버빌") | (coordinate["APT_NM"]=="용산푸르지오써밋") | (coordinate["APT_NM"]=="마포한강아이파크") | 
                    (coordinate["APT_NM"]=="아크로리버파크") | ((coordinate["APT_NM"]=="아이파크") & (coordinate["DONG"]=="삼성동"))]

,APT_NM,X_CODE,Y_CODE,USE_RQSTDT,target,is_test,GU,DONG
3092,아이파크,127.054362,37.514784,NaN,370000.0,0,강남구,삼성동
3093,아이파크,127.054362,37.514784,NaN,370000.0,0,강남구,삼성동
3094,아이파크,127.054362,37.514784,NaN,340000.0,0,강남구,삼성동
3095,아이파크,127.054362,37.514784,NaN,338000.0,0,강남구,삼성동
3096,아이파크,127.054362,37.514784,NaN,348000.0,0,강남구,삼성동
...,...,...,...,...,...,...,...,...
5925,아크로리버뷰신반포,127.008650,37.516620,NaN,NaN,1,서초구,잠원동
5926,아크로리버뷰신반포,127.008650,37.516620,NaN,NaN,1,서초구,잠원동
5927,아크로리버뷰신반포,127.008650,37.516620,NaN,NaN,1,서초구,잠원동
5928,아크로리버뷰신반포,127.008650,37.516620,NaN,NaN,1,서초구,잠원동


In [ ]:
lat1 = np.deg2rad(veryhard['경도'])
lat2 = np.deg2rad(veryhard['좌표X'])
lon1 = np.deg2rad(veryhard['위도'])
lon2 = np.deg2rad(veryhard['좌표Y'])
dlat = lat2 - lat1
dlon = lon1 - lon2
a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
c = 2 * np.arcsin(np.sqrt(a))